In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("C:/Users/SHAHZOR AHMED/OneDrive/Desktop/Full Stack Academy/DL/Tweets.csv")
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
nltk.download('stopwords')
def preprocess_text(text):
    text = text.lower()
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text
preprocess_text("My name is Rahul an the is")

[nltk_data] Downloading package stopwords to C:\Users\SHAHZOR
[nltk_data]     AHMED\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'name rahul'

In [4]:
pd.DataFrame(data['text'])

,text
0,@VirginAmerica What @dhepburn said.
1,@VirginAmerica plus you've added commercials t...
2,@VirginAmerica I didn't today... Must mean I n...
3,@VirginAmerica it's really aggressive to blast...
4,@VirginAmerica and it's a really big bad thing...
...,...
14635,@AmericanAir thank you we got on a different f...
14636,@AmericanAir leaving over 20 minutes Late Flig...
14637,@AmericanAir Please bring American Airlines to...
14638,"@AmericanAir you have my money, you change my ..."


In [5]:
data['text'] = data['text'].apply(preprocess_text)

In [6]:
pd.DataFrame(data['text'])

,text
0,@virginamerica @dhepburn said.
1,@virginamerica plus added commercials experien...
2,@virginamerica today... must mean need take an...
3,@virginamerica really aggressive blast obnoxio...
4,@virginamerica really big bad thing
...,...
14635,@americanair thank got different flight chicago.
14636,@americanair leaving 20 minutes late flight. w...
14637,@americanair please bring american airlines #b...
14638,"@americanair money, change flight, answer phon..."


In [7]:
tokenizer = Tokenizer(num_words = 10000, oov_token = "<OOV>")

In [8]:
tokenizer.fit_on_texts(data['text'])

In [9]:
sequences = tokenizer.texts_to_sequences(data['text'])

In [10]:
padded_sequences = pad_sequences(sequences, maxlen = 200, padding = 'post', truncating= 'post')
padded_sequences

array([[  35, 6648,  146, ...,    0,    0,    0],
       [  35,  464, 1070, ...,    0,    0,    0],
       [  35,   46,  710, ...,    0,    0,    0],
       ...,
       [   5,   28,  566, ...,    0,    0,    0],
       [   5,  289,   65, ...,    0,    0,    0],
       [   5,  221,  771, ...,    0,    0,    0]])

In [11]:
data['airline_sentiment'].value_counts()

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

In [12]:
labels = pd.get_dummies(data['airline_sentiment']).values
labels

array([[False,  True, False],
       [False, False,  True],
       [False,  True, False],
       ...,
       [False,  True, False],
       [ True, False, False],
       [False,  True, False]])

In [13]:
Sentiment_analysis_model = Sequential([
    Embedding(input_dim=10000, output_dim = 128, input_length = 200),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])
Sentiment_analysis_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\SHAHZOR AHMED\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [17]:
num_epochs = 2
history = Sentiment_analysis_model.fit(X_train, Y_train, epochs=num_epochs, validation_data=(X_test, Y_test), batch_size=32)

Epoch 1/2
366/366 ━━━━━━━━━━━━━━━━━━━━ 43s 118ms/step - accuracy: 0.9542 - loss: 0.1423 - val_accuracy: 0.7661 - val_loss: 0.9216
Epoch 2/2
366/366 ━━━━━━━━━━━━━━━━━━━━ 42s 115ms/step - accuracy: 0.9606 - loss: 0.1219 - val_accuracy: 0.7592 - val_loss: 1.1381


In [18]:
test_loss, test_acc = Sentiment_analysis_model.evaluate(X_test, Y_test)
print("Test Accuracy: ", test_acc)

92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.7564 - loss: 1.1466
Test Accuracy:  0.7592213153839111


In [22]:
# Function to preprocess and predict sentiment
def predict_sentiment(text, tokenizer, Sentiment_analysis_model):
    # Preprocess the text (tokenization and padding)
    text = preprocess_text(text)
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=200, padding='post', truncating='post')
    print(f"The padded seq is {padded_sequence}")

    # Make a prediction
    prediction = Sentiment_analysis_model.predict(padded_sequence)
    print(f"The prediction is {prediction}")

    # Get the sentiment with the highest probability
    sentiment_index = prediction.argmax(axis=1)[0]
    print(f" The sentiment index is {sentiment_index}")

    # Map index to sentiment label
    sentiment_labels = ['negative', 'neutral', 'positive']
    sentiment = sentiment_labels[sentiment_index]

    return sentiment


new_text = "I'm really happy with the service provided by the airline!"
predicted_sentiment = predict_sentiment(new_text, tokenizer, Sentiment_analysis_model)
print(f"The predicted sentiment is: {predicted_sentiment}")


The padded seq is [[  72  305   14 1084   47    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0